# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np 
import pandas as pd 
import requests 
import xlsxwriter  
import math 
from scipy.stats import percentileofscore as score 

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2418980797352,
 'week52high': 145.5,
 'week52low': 56.41,
 'week52change': 0.8533296224108146,
 'sharesOutstanding': 16907033486,
 'float': 0,
 'avg10Volume': 115435240,
 'avg30Volume': 115506158,
 'day200MovingAvg': 120.88,
 'day50MovingAvg': 133.8,
 'employees': 0,
 'ttmEPS': 3.32,
 'ttmDividendRate': 0.8069858542675179,
 'dividendYield': 0.005722032318076269,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-26',
 'nextEarningsDate': '2021-01-17',
 'peRatio': 42.65909326362978,
 'beta': 1.1745593314457845,
 'maxChangePercent': 56.40618100723043,
 'year5ChangePercent': 5.241725635126727,
 'year2ChangePercent': 2.7947547554624657,
 'year1ChangePercent': 0.8168785358151294,
 'ytdChangePercent': 0.07994078165805438,
 'month6ChangePercent': 0.5699808798893626,
 'month3ChangePercent': 0.25183309725776165,
 'month1ChangePercent': 0.08640730949005129,
 'day30ChangePercent': 0.0850243713593143,
 'day5ChangePercent': 0.1223792046393141}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.8168785358151294

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,123.700,0.418566,N/A
1,AAL,17.376,-0.436687,N/A
2,AAP,152.530,0.127092,N/A
3,AAPL,146.730,0.824961,N/A
4,ABBV,107.510,0.372058,N/A
...,...,...,...,...
500,YUM,102.630,0.0328516,N/A
501,ZBH,157.210,0.0795323,N/A
502,ZBRA,407.870,0.565734,N/A
503,ZION,44.960,0.0260157,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,38.080,2.16,N/A
1,FCX,26.430,1.31353,N/A
2,ALB,170.480,1.25578,N/A
3,NVDA,537.420,1.20417,N/A
4,PYPL,240.560,1.10664,N/A
5,LB,38.570,1.09893,N/A
6,ALGN,539.990,1.01969,N/A
7,WST,302.600,0.956695,N/A
8,SIVB,477.692,0.928943,N/A
9,AMD,92.780,0.915607,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:1000000
1000000


In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

c:\users\rkrit\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,38.080,2.16,514
1,FCX,26.430,1.31353,741
2,ALB,170.480,1.25578,115
3,NVDA,537.420,1.20417,36
4,PYPL,240.560,1.10664,81
5,LB,38.570,1.09893,508
6,ALGN,539.990,1.01969,36
7,WST,302.600,0.956695,64
8,SIVB,477.692,0.928943,41
9,AMD,92.780,0.915607,211


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [11]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        if hqm_dataframe.loc[row, change_col] == None:
            hqm_dataframe.loc[row, change_col] = 0.0

for row in hqm_dataframe.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'

        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
        
#Print the entire DataFrame    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,122.612,N/A,0.417104,0.865347,0.313733,0.685149,0.194248,0.619802,0.0607573,0.693069,N/A
1,AAL,17.440,N/A,-0.45583,0.0158416,0.381266,0.774257,0.323421,0.790099,-0.00860913,0.279208,N/A
2,AAP,152.480,N/A,0.123536,0.538614,0.0750935,0.239604,0.0441454,0.257426,-0.0191899,0.20396,N/A
3,AAPL,144.760,N/A,0.830242,0.972277,0.560414,0.90297,0.24712,0.710891,0.0879375,0.813861,N/A
4,ABBV,106.056,N/A,0.377527,0.839604,0.149508,0.382178,0.324366,0.792079,0.068653,0.740594,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,103.487,N/A,0.0334402,0.411881,0.15553,0.39604,0.0803598,0.332673,-0.0111938,0.259406,N/A
501,ZBH,159.640,N/A,0.0819801,0.471287,0.192028,0.477228,0.138977,0.49703,0.061611,0.69703,N/A
502,ZBRA,398.480,N/A,0.577186,0.928713,0.481932,0.867327,0.339447,0.807921,0.0488545,0.623762,N/A
503,ZION,46.210,N/A,0.0257047,0.4,0.410133,0.813861,0.436428,0.891089,0.0627509,0.706931,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False,inplace= True)
hqm_dataframe = hqm_dataframe[:51]
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,SIVB,475.760,N/A,0.916425,0.984158,1.12964,0.992079,0.685161,0.970297,0.293503,0.992079,0.984653
1,ALB,169.780,N/A,1.2164,0.99604,1.06677,0.986139,0.877911,0.986139,0.174939,0.944554,0.978218
2,VIAC,54.911,N/A,0.517335,0.90495,1.14497,0.994059,0.852215,0.984158,0.463176,1,0.970792
3,AMAT,103.460,N/A,0.739141,0.958416,0.767406,0.966337,0.786302,0.978218,0.242319,0.976238,0.969802
4,LRCX,523.400,N/A,0.848533,0.974257,0.627116,0.922772,0.618021,0.968317,0.157968,0.930693,0.94901
5,GM,49.650,N/A,0.551378,0.920792,1.01647,0.982178,0.456233,0.906931,0.261513,0.984158,0.948515
6,DISCA,44.060,N/A,0.354654,0.819802,0.932828,0.980198,1.01907,0.994059,0.434331,0.99802,0.94802
7,KLAC,300.820,N/A,0.747321,0.962376,0.650379,0.936634,0.53506,0.938614,0.171725,0.942574,0.94505
8,MOS,27.634,N/A,0.453543,0.881188,1.03793,0.984158,0.527581,0.934653,0.248866,0.980198,0.94505
9,FCX,26.444,N/A,1.32967,0.99802,1.11088,0.990099,0.568576,0.954455,0.0925192,0.831683,0.943564


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portfolio_input()

Enter the value of your portfolio:1000000


In [15]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

c:\users\rkrit\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,SIVB,475.760,41,0.916425,0.984158,1.12964,0.992079,0.685161,0.970297,0.293503,0.992079,0.984653
1,ALB,169.780,115,1.2164,0.99604,1.06677,0.986139,0.877911,0.986139,0.174939,0.944554,0.978218
2,VIAC,54.911,357,0.517335,0.90495,1.14497,0.994059,0.852215,0.984158,0.463176,1,0.970792
3,AMAT,103.460,189,0.739141,0.958416,0.767406,0.966337,0.786302,0.978218,0.242319,0.976238,0.969802
4,LRCX,523.400,37,0.848533,0.974257,0.627116,0.922772,0.618021,0.968317,0.157968,0.930693,0.94901
5,GM,49.650,394,0.551378,0.920792,1.01647,0.982178,0.456233,0.906931,0.261513,0.984158,0.948515
6,DISCA,44.060,445,0.354654,0.819802,0.932828,0.980198,1.01907,0.994059,0.434331,0.99802,0.94802
7,KLAC,300.820,65,0.747321,0.962376,0.650379,0.936634,0.53506,0.938614,0.171725,0.942574,0.94505
8,MOS,27.634,709,0.453543,0.881188,1.03793,0.984158,0.527581,0.934653,0.248866,0.980198,0.94505
9,FCX,26.444,741,1.32967,0.99802,1.11088,0.990099,0.568576,0.954455,0.0925192,0.831683,0.943564


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [19]:
writer.save()